<a href="https://colab.research.google.com/github/Bayrem-ben/colab.github.io/blob/main/Copy_of_BERT_arabert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align=center> BERT-for-article-Classification</h2>




In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Check GPU Availability

Check if your Colab notebook is configured to use Graphical Processing Units (GPUs). If zero GPUs are available, check if the Colab notebook is configured to use GPUs (Menu > Runtime > Change Runtime Type).

![Hardware Accelerator Settings](https://drive.google.com/uc?id=1qrihuuMtvzXJHiRV8M7RngbxFYipXKQx)


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



### Install TensorFlow and TensorFlow Model Garden

In [ ]:
import tensorflow as tf
print(tf.version.VERSION)

2.6.0


In [ ]:
!pip install -q tensorflow==2.3.0

     |████████████████████████████████| 320.4 MB 16 kB/s 
     |████████████████████████████████| 2.9 MB 46.1 MB/s 
     |████████████████████████████████| 20.1 MB 138 kB/s 
     |████████████████████████████████| 459 kB 82.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 2650, done.
remote: Counting objects: 100% (2650/2650), done.
remote: Compressing objects: 100% (2311/2311), done.
remote: Total 2650 (delta 506), reused 1388 (delta 306), pack-reused 0
Receiving objects: 100% (2650/2650), 34.02 MiB | 40.60 MiB/s, done.
Resolving deltas: 100% (506/506), done.
Note: checking out '400d68abbccda2f0f6609e3a924467718b144233'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [ ]:
# install requirements to use tensorflow/models repository
!pip install -Uqr models/official/requirements.txt
# you may have to restart the runtime afterwards

     |████████████████████████████████| 7.6 MB 5.2 MB/s 
     |████████████████████████████████| 202 kB 76.1 MB/s 
     |████████████████████████████████| 15.7 MB 197 kB/s 
     |████████████████████████████████| 11.3 MB 52.8 MB/s 
     |████████████████████████████████| 296 kB 71.7 MB/s 
     |████████████████████████████████| 99 kB 12.5 MB/s 
     |████████████████████████████████| 28.5 MB 41 kB/s 
     |████████████████████████████████| 213 kB 55.1 MB/s 
     |████████████████████████████████| 4.0 MB 32.0 MB/s 
     |████████████████████████████████| 1.1 MB 73.9 MB/s 
     |████████████████████████████████| 352 kB 92.2 MB/s 
     |████████████████████████████████| 1.2 MB 82.2 MB/s 
     |████████████████████████████████| 10.3 MB 25.9 MB/s 
     |████████████████████████████████| 37.1 MB 54 kB/s 
     |████████████████████████████████| 596 kB 79.6 MB/s 
     |████████████████████████████████| 3.1 MB 23.8 MB/s 
     |████████████████████████████████| 75 kB 5.4 MB/s 
     |████████████

## Restart the Runtime

**Note** 
After installing the required Python packages, you'll need to restart the Colab Runtime Engine (Menu > Runtime > Restart runtime...)

![Restart of the Colab Runtime Engine](https://drive.google.com/uc?id=1xnjAy2sxIymKhydkqb0RKzgVK9rh3teH)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

In [ ]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF Version:  2.6.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel("/content/drive/MyDrive/press/article_ar_excel.xlsx")
#df = pd.read_csv('/content/drive/MyDrive/press/article_clean.csv')
df.shape

(1282, 7)

In [ ]:
df.tail()

,Article,Type,Unnamed: 2,Unnamed: 3,Unnamed: 4,Category,Unnamed: 6
1277,الشركة الجهوية للنقل القوافل قفصة اعلان تمديد ...,طلب العروض,NaN,NaN,NaN,اقتصاد,NaN
1278,استشارة عدد وع بس اقتناء مواد ومستلزمات تنظيف...,الشركة والاستشارات,NaN,NaN,NaN,ثقافة,NaN
1279,إعلان بيع بالمزاد العلني,بيع بالمزاد العلني,NaN,NaN,NaN,تكنولوجيا,NaN
1280,أن اختيار المياه المعدنية للشرب يساعد على إضاف...,صحة,NaN,NaN,NaN,صحة,NaN
1281,تحسين صحة القلب من الشائع لدى الجميع أن اللحوم...,صحة,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Category', 'Unnamed: 6'])

In [ ]:
df['Article'].drop_duplicates(inplace=True)
df['Article'].duplicated().sum()

10

In [ ]:
df.plot(kind='hist', title='distribution')

## Task 4: Create tf.data.Datasets for Training and Evaluation

In [ ]:
df_train, df_test = train_test_split(df, random_state=20, train_size=0.8, shuffle=True)

In [ ]:
df_train.shape, df_test.shape

((1025, 4), (257, 4))

In [ ]:
with tf.device('/cpu:0'):
  data = tf.data.Dataset.from_tensor_slices((df['Article_cleaned'].values, df['encodedLabel'].values))

  for text, label in data.take(1):
    print(text)
    print(label)

tf.Tensor(b'\xd8\xa3\xd8\xa8\xd9\x88\xd8\xb8\xd8\xa8\xd9\x8a \xd8\xa5\xd9\x8a\xd9\x85\xd8\xa7\xd9\x86 \xd8\xb3\xd8\xb1\xd9\x88\xd8\xb1 \xd9\x8a\xd8\xb4\xd9\x83\xd9\x84   \xd8\xa7\xd9\x84\xd9\x85\xd8\xb1\xd8\xad\xd9\x88\xd9\x85 \xd8\xb9\xd8\xa8\xd8\xaf\xd8\xa7\xd9\x84\xd8\xac\xd9\x84\xd9\x8a\xd9\x84 \xd8\xa7\xd9\x84\xd9\x81\xd9\x87\xd9\x8a\xd9\x85 \xd9\x88\xd8\xb9\xd8\xa7\xd8\xa6\xd9\x84\xd8\xaa\xd9\x87   \xd9\x88\xd8\xa7\xd8\xad\xd8\xaf\xd8\xa7 \xd8\xa7\xd9\x84\xd9\x85\xd8\xb4\xd8\xb1\xd9\x88\xd8\xb9\xd8\xa7\xd8\xaa \xd8\xa7\xd9\x84\xd9\x85\xd9\x85\xd9\x8a\xd8\xb2\xd8\xa9 \xd8\xa7\xd9\x84\xd8\xb9\xd9\x85\xd9\x84 \xd8\xa7\xd9\x84\xd8\xae\xd9\x8a\xd8\xb1\xd9\x8a \xd9\x88\xd9\x8a\xd8\xb2\xd9\x8a\xd8\xaf \xd8\xa3\xd8\xb1\xd8\xb6 \xd8\xaf\xd9\x88\xd9\x84\xd8\xa9 \xd8\xa7\xd9\x84\xd8\xa5\xd9\x85\xd8\xa7\xd8\xb1\xd8\xa7\xd8\xaa \xd8\xaa\xd9\x86\xd8\xa7\xd8\xb2\xd9\x84 \xd8\xa3\xd8\xa8\xd9\x86\xd8\xa7\xd8\xa1 \xd8\xa7\xd9\x84\xd9\x85\xd8\xb1\xd8\xad\xd9\x88\xd9\x85 \xd8\xb9\xd8\xa8\xd8\xaf\xd8

## Task 5: Download a Pre-trained AraBERT Model from TensorFlow Hub

In [ ]:
!git clone https://github.com/aub-mind/arabert.git
!pip install pyarabic
!pip install farasapy
!pip install transformers
!pip install nltk

In [ ]:
"""
from arabert.preprocess import ArabertPreprocessor

# Label categories
T_list = sorted(df['encodedLabel'].drop_duplicates())
label_list = [T_list]
max_seq_length = 512 # maximum length of (token) input sequences
train_bash_size = 32

# Get BERT layer and tokenizer:
model_name="bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)

arabert_prep.preprocess(df) 
"""

In [ ]:
#apply filter stop_word
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.utils import shuffle
import re
import string
import sys
import argparse
from arabert.preprocess import ArabertPreprocessor
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

def stop_word():
    Stop_w = list(set(stopwords.words('arabic')))
    Tunisia_stop_words = '/content/drive/MyDrive/press/tunisian_stop_words.txt'
    with open(Tunisia_stop_words, 'r+') as f:
        lists = f.readlines() #readlines  .rstrip('\n')
    lines = []    
    for line in lists:
        stripped_line = line.rstrip('\n')
        lines.append(stripped_line)
    stop_list = lines + Stop_w
    stop_words = sorted(set(stop_list), key=lambda x:stop_list.index(x))
    return stop_words

def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                            َ    | # Fatha
                            ً    | # Tanwin Fath
                            ُ    | # Damma
                            ٌ    | # Tanwin Damm
                            ِ    | # Kasra
                            ٍ    | # Tanwin Kasr
                            ْ    | # Sukun
                            ـ     # Tatwil/Kashida
                        """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean_text(text):
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    text = "".join([word for word in text if word not in punctuations_list])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    stop_words = stop_word()
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

def prep_txt_arabert(text):
    model_name = "aubmindlab/bert-base-arabertv2"
    arabert_prep = ArabertPreprocessor(model_name=model_name)
    outputs = arabert_prep.preprocess(text)
    txt_clean = clean_text(outputs)
    #data['cleaned_text'] = data[txt_column].apply(clean_text)
    #for row in data.itertuples():
    text_clean = re.sub(r'[\W\s]', ' ', txt_clean) #row.cleaned_text))
    return text_clean

def prep_arabert(data, txt_column, Label_name):
    model_name = "aubmindlab/bert-base-arabertv2"
    arabert_prep = ArabertPreprocessor(model_name=model_name)
    List = []
    cl_list = []
    for txt in data[txt_column]:
    #for txt in data.itertuples():
        outputs = arabert_prep.preprocess(txt)
        txt_clean = clean_text(outputs)
        cl_list.append(txt_clean)
    data['cleaned_text'] = cl_list
    for row in data.itertuples():
        List.append(re.sub(r'[\W\s]', ' ', row.cleaned_text)) #row.cleaned_text)) txt_clean
    data['Article_cleaned'] = List
    data = data.drop(columns=['cleaned_text'])
    data = shuffle(data)
    label_encoder = LabelEncoder()
    data['encodedLabel'] = label_encoder.fit_transform(data[Label_name])
    return data

data = prep_arabert(df, 'Article', 'Type')

[2021-10-16 21:25:21,928 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[2021-10-16 21:25:27,107 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


إعلان بيع مزاد علني مر حضور ديواني


In [ ]:
data

,Article,Type,Article_cleaned,encodedLabel
1149,تعلم جمعية نادي الصحة الكاراتي بالمهدية أنها س...,إعلان,تعلم جمعي نادي صح كاراتي مهدي تعقد جلس خارق إث...,1
197,وزارة الشؤون المحليةوالبيئة الديوان الوطني للت...,إعلان,وزار شؤون محليةوالبيئ ديوان وطني تطهير إعلان ب...,1
353,['القاهرة:«الخليج» عاقبت محكمة جنايات شمال الق...,سياسة,قاهر خليج عاقب محكم جناي ات قاهر حازم صلاح...,10
1234,گراء أصل تجاري . وكالة حرة بمقتضى عقد بخط اليد...,أصل تجاري,أصل تجاري وكال حر مقتضى عقد خط يد تاريخ مسجل ق...,0
156,الجمهورية التونسية وزارة الفلاحة والموارد الم...,إعلان,جمهوري تونسي وزار فلاح موارد مائي صيد بحري مند...,1
...,...,...,...,...
873,['رأس الخيمة: «الخليج» نقل السفير الإثيوبي الس...,صحة,رأس خيم خليج نقل سفير إثيوبي سابق أستراليا...,11
511,['أبوظبي «الخليج»: انخفض مؤشر سوق الإمارات الم...,اقتصاد,أبوظبي خليج انخفض مؤشر سوق إمار ات مالي صا...,2
568,['عقدت اللجنة المنظمة للدورة الثالثة لمعرض تعا...,ثقافة,عقد لجن منظم دور معرض تعابير إماراتي رؤي تتحقق...,7
529,['أظهرت نتائج بعض الشركات المساهمة والبنوك الو...,اقتصاد,أظهر نتائج شرك ات مساهم بنوك وطني تباين أرباح ...,2


In [ ]:
data.to_csv(r'/content/drive/MyDrive/press/article_clean.csv', index = False)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

tokenized_text = tokenizer.tokenize("إعلان بيع بالمزاد العلني للمرة الثالثة بحضور الديوانية")
clean_text = clean_text(tokenized_text)
tokenized = tokenizer.tokenize(clean_text)
input = tokenizer.convert_tokens_to_ids(tokenized)
print(tokenized_text)
print(clean_text)
print(tokenized)
print(input)

#model = hub.load('https://tfhub.dev/callmemehdi/AraBERT/1')
#outputs = model(tf.convert_to_tensor([input], dtype=tf.int32))
#print(outputs)

['إعلان', 'بيع', 'بالم', '##زاد', 'العل', '##ني', 'ل', '##لم', '##رة', 'ال', '##ثالث', '##ة', 'بحضور', 'الدي', '##وانية']
إعلانبيعبالم # # زادالعل # # نيل # # # # رةال # # # # ةبحضورالدي # # وانية
['إعلان', '##بيع', '##بال', '##م', '#', '#', 'زاد', '##الع', '##ل', '#', '#', 'نيل', '#', '#', '#', '#', 'رة', '##ال', '#', '#', '#', '#', 'ة', '##بحضور', '##الدي', '#', '#', 'واني', '##ة']
[1461, 2045, 3506, 199, 38, 38, 3139, 7042, 196, 38, 38, 3187, 38, 38, 38, 38, 32861, 292, 38, 38, 38, 38, 141, 31041, 50076, 38, 38, 21957, 251]


In [ ]:
# convert to tf number better optimization and all



### **Install library**

In [ ]:
!git clone https://github.com/aub-mind/arabert.git
!pip install pyarabic
!pip install farasapy
!pip install transformers
!pip install nltk

In [ ]:
!pip install PyArabic farasapy fast-bert

In [ ]:
!pkill "java"

In [ ]:
#Checking for GPU
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## **Youtube**

In [ ]:
import pandas as pd
#from py4j.java_gateway import JavaGateway
from farasa.segmenter import FarasaSegmenter
#from arabert.preprocess_arabert import preprocess
from arabert.preprocess import ArabertPreprocessor
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer
import re
import string
import sys
import argparse

farasa_segmenter = FarasaSegmenter(interactive=True)

In [ ]:
### follow youtube
df_AJGT = pd.read_csv('/content/drive/MyDrive/press/article_clean.csv')

In [ ]:
DATA_COLUMN = 'Article_cleaned'
LABEL_COLUMN = 'encodedLabel'

df_AJGT = df_AJGT[['Article_cleaned', 'encodedLabel']]
df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]

In [ ]:
#df_AJGT[DATA_COLUMN] = df_AJGT[DATA_COLUMN].apply(lambda x: ArabertPreprocessor(model_name=model_name).preprocess(x))
from transformers import AutoTokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.utils import shuffle
import re
import string
import sys
import argparse
from arabert.preprocess import ArabertPreprocessor
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder


#tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
#tokenized = tokenizer.tokenize(df_AJGT[DATA_COLUMN])

List = []
cl_list = []
model_name="bert-large-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)
for txt in df_AJGT[DATA_COLUMN]:
#for txt in df_AJGT.itertuples():
    outputs = arabert_prep.preprocess(txt)
    print(outputs)
    List.append(outputs)

df_AJGT[DATA_COLUMN] = List

In [ ]:
df_AJGT.head()

,Article_cleaned,encodedLabel
0,تعلم جمعي نادي صح كار +ات +ي مهدي تعقد جلس خار...,1
1,و+ زار شؤون محليةوالبيئ ديوان وطني تطهير إعلان...,1
2,قاهر خليج عاقب محكم جناي ات قاهر حازم صلاح إسم...,10
3,أصل تجاري و+ كال حر مقتضى عقد خط يد تاريخ مسجل...,0
4,جمهوري تونسي و+ زار ف+ لاح موارد مائي صيد بحري...,1


In [ ]:
train_df = pd.DataFrame({
    'id':range(len(df_AJGT)),
    'label':df_AJGT["encodedLabel"],
    'alpha':['a']*df_AJGT.shape[0],
    'text': df_AJGT["Article_cleaned"].replace(r'\n', ' ', regex=True)
})

In [ ]:
!mkdir data
train_df.to_csv("data/train.tsv",index=False,columns=train_df.columns,sep='\t',header=False)

# **Model Finetuning**

In [ ]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json
import csv
import sys
from io import open
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import numpy as np

import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers import BertPreTrainedModel
from transformers import BertModel
from transformers import WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer                                  
from transformers import AdamW, get_linear_schedule_with_warmup

from arabert.preprocess import never_split_tokens

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from transformers import BertForSequenceClassification

#https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_bert.py#L1107

In [ ]:
csv.field_size_limit(2147483647)

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_predict_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir , train_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, train_file_name)), "train")

    def get_dev_examples(self, data_dir, dev_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, dev_file_name)), "dev")

    def get_predict_examples(self, data_dir, train_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, train_file_name)), "predict")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []

        if(set_type!="predict"):
            for (i, line) in enumerate(lines):
                guid = "%s-%s" % (set_type, i)
                text_a = line[3]
                label = line[1]
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        else:
            for (i, line) in enumerate(lines):
                guid = "%s-%s" % (set_type, i)
                text_a = line[3]
                label = '0'
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True, sep_token_extra=False):
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id, sep_token_extra = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3". " -4" for RoBERTa.
        special_tokens_count = 4 if sep_token_extra else 3
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - special_tokens_count)
    else:
        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens_a) > max_seq_length - special_tokens_count:
            tokens_a = tokens_a[:(max_seq_length - special_tokens_count)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)


    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)
  

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, sep_token_extra=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id, sep_token_extra) for example in examples]

    process_count = cpu_count()

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=500), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

In [ ]:
def load_and_cache_examples(task, tokenizer, mode="train"):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    cached_features_file = os.path.join(args['data_dir'], f"cached_{mode}_{args['cache_dir']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        if mode=="train":
          examples=processor.get_train_examples(args['data_dir'],args['train_file_name']);
        if mode=="dev":
          examples=processor.get_dev_examples(args['data_dir'],args['dev_file_name']);
        if mode=="predict":
          examples=processor.get_predict_examples(args['data_dir'],args['pred_file_name'])
        
        if __name__ == "__main__":
            features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
                cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
                cls_token=tokenizer.cls_token,
                cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
                sep_token=tokenizer.sep_token,
                sep_token_extra=bool(args['model_type'] in ['roberta']),           # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
                pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [ ]:
def train(model, tokenizer):
    tb_writer = SummaryWriter(log_dir=f"{args['log_dir']}/{args['notes']}/seq_len_{args['max_seq_length']}_BS_{args['train_batch_size']}_lr_{args['learning_rate']}")
    
    train_dataset = load_and_cache_examples(args['task_name'], tokenizer , mode="train")
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
    num_training_steps = len(train_dataloader) * args['num_train_epochs']
    
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']

    #model.named_parameters(): returns the name of the parameter as well as the parameter itself
    #'bias', 'LayerNorm.weight' should have zero decay
    optimizer_grouped_parameters = [
        {'params': [param for name, param in model.named_parameters() if not any(nd in name for nd in no_decay)],
         'weight_decay': args['weight_decay'],
        },
        {'params': [param for name, param in model.named_parameters() if any(nd in name for nd in no_decay)],
         'weight_decay': 0.0,
        }
        ]
    
    #correct_bias (bool): can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], correct_bias=True)
    # PyTorch scheduler
    # https://huggingface.co/transformers/main_classes/optimizer_schedules.html
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=num_training_steps)     

    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Total optimization steps = %d", num_training_steps)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()

    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")    
    for epoch in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):

            #model.train() tells your model that you are training the model.
            #So effectively layers like dropout, batchnorm etc. which behave different on the train and test procedures know what is going on and hence can behave accordingly
            model.train()

            batch = tuple(t.to(device) for t in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}

            outputs = model(**inputs)

            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            print("\r%f" % loss, end='')

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

            tr_loss += loss.item() #trainning loss
                
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            global_step += 1

            if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                # Log metrics
                tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args['logging_steps'], global_step)
                logging_loss = tr_loss

            if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                # Save model checkpoint
                output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                model_to_save.save_pretrained(output_dir)
                logger.info("Saving model checkpoint to %s", output_dir)

        if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
            results = evaluate(model, tokenizer)
            for key, value in results.items():
                tb_writer.add_scalar('eval_{}'.format(key), value, epoch)

    return global_step, tr_loss / global_step

In [ ]:
from sklearn.metrics import (mean_squared_error, matthews_corrcoef, confusion_matrix,
                             f1_score, precision_score , recall_score , accuracy_score)
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args['data_dir'],args['dev_file_name'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):

    assert len(preds) == len(labels)
    print(classification_report(labels,preds))
    print(confusion_matrix(labels,preds))

    f1_Positive = f1_score(labels,preds,pos_label=1,average='binary')
    f1_Negative = f1_score(labels,preds,pos_label=0,average='binary')
    macro_f1 = f1_score(labels,preds,average='macro')
    macro_precision = precision_score(labels,preds,average='macro')
    macro_recall = recall_score(labels,preds,average='macro')
    acc = accuracy_score(labels,preds)
    return {
        'f1_pos': f1_Positive,
        'f1_neg': f1_Negative,
        'macro_f1' : macro_f1, 
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'accuracy': acc
    }
    #return get_eval_report(labels, preds)

def evaluate(model, tokenizer, prefix=""):
    
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, mode="dev")
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)

    #result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results = compute_metrics(EVAL_TASK, preds, out_label_ids)
    
    return results#, wrong

In [ ]:
def predict(model, tokenizer, prefix=""):
    pred_output_dir = args['output_dir']
  
    results={}
    PRED_TASK = args['task_name']
  
    pred_dataset = load_and_cache_examples(PRED_TASK, tokenizer, mode='predict')
    if not os.path.exists(pred_output_dir):
      os.makedirs(pred_output_dir)
    
  
    pred_sampler = SequentialSampler(pred_dataset)
    pred_dataloader = DataLoader(pred_dataset, sampler=pred_sampler, batch_size=args['eval_batch_size'])
  
    logger.info("***** Running prediction {} *****".format(prefix))
    logger.info("  Num examples = %d", len(pred_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
  
    preds = None
    for batch in pred_dataloader:
      with torch.no_grad():
          batch = tuple(t.to(device) for t in batch)
          inputs = {'input_ids': batch[0],'attention_mask': batch[1],'token_type_ids': batch[2],'labels': batch[3]}
        
          outputs = model(**inputs)
          _, logits = outputs[:2]
    if preds is None:
        preds = logits.detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

    preds = np.argmax(preds, axis=1)
      
    return preds

In [ ]:
args = {
    'data_dir': 'data/',
    'train_file_name': 'train.tsv',
    'dev_file_name': 'dev.tsv',
    'pred_file_name': 'dev.tsv',
    'model_type':  'bert',
    'model_name': 'aubmindlab/bert-base-arabert',
    'task_name': 'binary',
    'output_dir': 'outputs_bert/',
    'cache_dir': 'cache',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 16,
    'eval_batch_size': 32,
    'num_train_epochs': 1,
    'weight_decay': 0,
    'learning_rate': 2e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,
    'log_dir':'/logs',
    'logging_steps': 0,
    'evaluate_during_training': True,
    'save_steps': 90,
    'eval_all_checkpoints': True,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'notes': 'AJGT_arabert'
}
with open('args.json', 'w') as f:
    json.dump(args, f)

!mkdir ./{args['log_dir']}

In [ ]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    #'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    #'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    #'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [ ]:
num_labels = df_AJGT['encodedLabel'].drop_duplicates()

#https://github.com/huggingface/transformers/blob/master/src/transformers/configuration_bert.py#L52
config = config_class.from_pretrained(args['model_name'], num_labels=len(num_labels), finetuning_task=args['task_name'])

model = model_class.from_pretrained(args['model_name'], output_attentions=True)
model.to(device)

#https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_bert.py#L119
tokenizer = tokenizer_class.from_pretrained(args['model_name'],
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)

In [ ]:
task = args['task_name']

if task in processors.keys() and task in output_modes.keys():
    processor = processors[task]()
    label_list = processor.get_labels()
    num_labels = len(label_list)
else:
    raise KeyError(f'{task} not found in processors or in output_modes. Please check utils.py.')

In [ ]:
!rm -rf outputs_bert

In [ ]:
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')
#tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

global_step, tr_loss = train(model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

## Task 10: Fine-Tune BERT for Text Classification

In [ ]:
# Building the model
def create_model():
  input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids'),
  input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask'),
  input_type_ids =tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids'),

  bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)
  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

  drop = tf.keras.layers.Dropout(0.4)(pooled_output)
  output = tf.keras.layers.Dense(1, activation='softmax', name="output") (drop)
  
  model = tf.keras.Model(
      input={'input_word_ids': input_word_ids, 'input_mask': input_mask, 'input_type_ids': input_type_ids},
      output=output)
  return model

In [ ]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss=tf.keras.losses.sparse_categorical_crossentropy(),
              metrics=[tf.keras.metrics.categorical_accuracy()])
model.summary

In [ ]:
tf.keras.utils.plot_model(model=model, show_shapes=True, dpi=70)

In [ ]:
# Train model
epochs = 4
history = model.fit(data, validation_data=0.8, epochs=epochs, verbose=1)

## Task 11: Evaluate the BERT Text Classification Model

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()